<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=1b8d80c5e04b8da6e0dbf0a55200a30178edf7840b8f3c1dfc785e49818181bb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 16:06:12
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -1.42 L (-0.99%)
Current PnL: -22.62 L (-14.82%)
CY Booked + Current PnL: -8.73 L (-5.72%)
-------------------
Total profit:  1.49 L
Total loss:  -24.11 L
-------------------
Total Booked + Current PnL: 18.25 L (14.53%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.03 L (61.49%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.49%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,-0.01,-14.23,16.70,0.09,14434.0,-14343.0,86434.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.56,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-1.97,-8.99,9.90,0.02,15728.0,-15690.0,158869.0,142.19,71.0,M-SC,4.21,234.0,-1.00,1.11,38.19,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,-1.13,9.19,9.61,19.69,20120.0,17628.0,209370.0,-1.49,54.0,X-MC,3.57,78.0,0.88,1.46,16.62,XY25,NTT,AC
43,ITC,452.0,0.59,-0.35,10.83,10.44,21599.0,-699.0,199439.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.19,X40,NTT,FMCG
53,NESTLEIND,1377.0,0.12,12.91,7.52,21.40,22536.0,34258.0,299684.0,4.80,63.0,X-LC,7.96,12.0,1.52,2.09,20.41,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,216.00,3.38,-19.66,53.14,23.03,76453.0,-35200.0,143871.0,55.70,48.0,H-SC,11.69,167.0,-0.46,1.00,42.75,XR,NTT,BANKS
40,INDUSINDBK,1800.00,2.07,-37.50,112.63,32.89,54347.0,-28954.0,48253.0,-695.74,64.0,L-MC,5.38,259.0,-0.53,0.34,32.89,XR,NTT,BANKS
14,CAMPUS,393.00,1.14,-11.52,50.63,33.28,75161.0,-19323.0,148452.0,-28.70,36.0,M-SC,2.96,222.0,-0.26,1.04,15.06,XY24,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.14,2.44,21.12,24.08,27467.0,3101.0,130051.0,-5.81,44.0,X-SC,4.51,89.0,0.11,0.91,22.54,XY24,NTT,HEALTHCARE
54,PAGEIND,51605.08,0.83,-5.01,32.71,26.07,50877.0,-8200.0,155540.0,-30.06,35.0,X-MC,8.69,55.0,-0.16,1.09,0.83,X40,ATH,APPARELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-3.86,-34.72,72.96,12.90,104060.0,-75870.0,142626.0,110.34,60.0,H-SC,15.11,161.0,-0.73,1.00,18.59,OX40N,NTT,FINANCE
72,TANLA,1943.92,-3.42,-34.72,236.55,119.69,418093.0,-94015.0,176746.0,-34.29,30.0,H-SC,12.48,156.0,-0.22,1.23,39.63,AR,ATH,IT
48,KPIGREEN,730.66,-3.23,-9.62,62.59,46.95,70883.0,-12048.0,113249.0,-16.60,30.0,H-SC,7.38,154.0,-0.17,0.79,41.87,MH,ATH,POWER
47,KANSAINER,340.00,-2.91,-22.30,46.05,13.47,96484.0,-60147.0,209520.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.46,6.53,XY24,NTT,PAINTS
69,STARHEALTH,761.00,-2.80,-4.91,54.86,47.26,146653.0,-13801.0,267322.0,29.82,49.0,H-SC,15.61,171.0,-0.09,1.87,43.22,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.09,0.63,105.42,106.72,130842.0,779.0,124115.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.16,AR,ATH,MISC
85,WIPRO,420.00,-0.72,0.28,71.79,72.27,121986.0,473.0,169921.0,-13.79,52.0,M-LC,5.05,99.0,0.00,1.19,6.47,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.22,-3.98,116.53,107.91,165199.0,-5879.0,141765.0,-21.77,36.0,M-SC,10.20,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.97,-8.99,9.90,0.02,15728.0,-15690.0,158869.0,142.19,71.0,M-SC,4.21,234.0,-1.00,1.11,38.19,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.01,-14.23,16.70,0.09,14434.0,-14343.0,86434.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.56,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.91,-22.30,46.05,13.47,96484.0,-60147.0,209520.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.46,6.53,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.12,-23.25,30.28,-0.01,29702.0,-29715.0,98090.0,19965.22,62.0,M-SC,1.54,233.0,-1.00,0.69,11.07,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.72,0.28,71.79,72.27,121986.0,473.0,169921.0,-13.79,52.0,M-LC,5.05,99.0,0.00,1.19,6.47,XR,NTT,IT
38,INDIAMART,4810.62,-2.09,0.63,105.42,106.72,130842.0,779.0,124115.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.16,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.09,0.63,105.42,106.72,130842.0,779.0,124115.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.16,AR,ATH,MISC
85,WIPRO,420.00,-0.72,0.28,71.79,72.27,121986.0,473.0,169921.0,-13.79,52.0,M-LC,5.05,99.0,0.00,1.19,6.47,XR,NTT,IT
25,FINCABLES,1641.55,-1.22,-3.98,116.53,107.91,165199.0,-5879.0,141765.0,-21.77,36.0,M-SC,10.20,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.64,-5.41,92.32,81.92,100248.0,-6210.0,108588.0,-44.55,31.0,M-SC,6.18,236.0,-0.06,0.76,28.55,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-0.40,-2.53,39.15,35.63,79972.0,-5304.0,204270.0,-16.30,32.0,H-MC,4.76,119.0,-0.07,1.43,13.72,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.89,-35.47,109.52,35.20,90938.0,-45637.0,83033.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-1.59,-16.17,70.96,43.32,96889.0,-26332.0,136540.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.95,4.84,X40N,ATH,ELECTRICAL
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
46,JSWINFRA,342.00,-2.30,-9.07,26.71,15.23,48661.0,-18165.0,182182.0,-22.32,25.0,X-MC,6.38,66.0,-0.37,1.27,20.26,X40N,NTT,REALTY
26,GILLETTE,11206.78,-1.52,-2.21,35.40,32.42,87896.0,-5601.0,248295.0,-19.56,29.0,X-SC,6.30,85.0,-0.06,1.73,12.56,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
43,ITC,452.00,0.59,-0.35,10.83,10.44,21599.0,-699.0,199439.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.19,X40,NTT,FMCG
36,ICICIPRULI,790.00,-1.21,1.28,29.32,30.97,52481.0,2256.0,178994.0,-22.52,49.0,X-MC,1.49,75.0,0.04,1.25,14.00,X40,ATH,INSURANCE
34,HONAUT,58357.33,-1.59,-16.17,70.96,43.32,96889.0,-26332.0,136540.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.95,4.84,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-1.15,-23.09,113.43,64.15,207588.0,-54941.0,183010.0,-55.00,44.0,X-SC,1.85,82.0,-0.26,1.28,2.66,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.89,-35.47,109.52,35.20,90938.0,-45637.0,83033.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.83,-13.68,71.99,48.45,124592.0,-27437.0,173069.0,-19.87,29.0,X-MC,7.29,64.0,-0.22,1.21,0.00,X40N,ATH,FINANCE
56,QUESS,424.00,-2.75,-29.30,102.05,42.84,46899.0,-19049.0,45957.0,-53.72,36.0,X-SC,37.16,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
54,PAGEIND,51605.08,0.83,-5.01,32.71,26.07,50877.0,-8200.0,155540.0,-30.06,35.0,X-MC,8.69,55.0,-0.16,1.09,0.83,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-2.75,-29.30,102.05,42.84,46899.0,-19049.0,45957.0,-53.72,36.0,X-SC,37.16,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-1.40,-47.10,187.92,52.31,144302.0,-68371.0,76789.0,-43.66,30.0,X-SC,3.45,91.0,-0.47,0.54,2.79,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.89,-35.47,109.52,35.20,90938.0,-45637.0,83033.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.14,2.44,21.12,24.08,27467.0,3101.0,130051.0,-5.81,44.0,X-SC,4.51,89.0,0.11,0.91,22.54,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-2.27,-3.89,26.51,21.58,35705.0,-5454.0,134685.0,-51.54,35.0,X-MC,7.21,56.0,-0.15,0.94,19.24,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.18,-14.39,39.34,19.28,116508.0,-49800.0,296156.0,-25.24,63.0,X-LC,7.77,1.0,-0.43,2.07,9.48,X40,ATH,IT
41,INFY,2275.00,0.55,5.18,47.25,54.87,155493.0,16197.0,329085.0,-18.16,60.0,X-LC,3.31,2.0,0.10,2.30,13.89,X40,BTT,IT
76,TMPV,600.00,0.68,-14.16,65.63,42.18,155723.0,-39136.0,237274.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.68,XY24,NTT,AUTO
82,VBL,671.64,-0.85,-9.57,50.04,35.67,142915.0,-30241.0,285601.0,-20.30,41.0,X-LC,6.13,4.0,-0.21,1.99,2.95,X40N,ATH,FMCG
43,ITC,452.00,0.59,-0.35,10.83,10.44,21599.0,-699.0,199439.0,-40.34,49.0,X-LC,0.96,5.0,-0.03,1.39,5.19,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.66,-12.76,108.24,81.67,88587.0,-11967.0,81843.0,7361.11,52.0,L-SC,18.39,271.0,-0.14,0.57,58.49,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.89,-12.96,60.32,39.54,53112.0,-13108.0,88051.0,-31.96,66.0,M-SC,6.87,220.0,-0.25,0.62,24.16,AR,ATH,AUTO
51,MASFIN,398.61,-0.41,-2.92,25.72,22.05,24464.0,-2865.0,95115.0,-16.44,58.0,H-SC,7.06,164.0,-0.12,0.66,38.26,XR,ATH,FINANCE
70,SURYODAY,216.00,3.38,-19.66,53.14,23.03,76453.0,-35200.0,143871.0,55.70,48.0,H-SC,11.69,167.0,-0.46,1.00,42.75,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-1.97,-8.99,9.90,0.02,15728.0,-15690.0,158869.0,142.19,71.0,M-SC,4.21,234.0,-1.00,1.11,38.19,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-1.97,-8.99,9.90,0.02,15728.0,-15690.0,158869.0,142.19,71.0,M-SC,4.21,234.0,-1.00,1.11,38.19,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.59,0.55,20.29,20.95,51362.0,1373.0,253141.0,-0.49,76.0,X-LC,12.12,31.0,0.03,1.77,36.38,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,2.07,-37.50,112.63,32.89,54347.0,-28954.0,48253.0,-695.74,64.0,L-MC,5.38,259.0,-0.53,0.34,32.89,XR,NTT,BANKS
13,BSOFT,831.70,-1.90,-30.85,118.07,50.79,114831.0,-43396.0,97257.0,-11.48,53.0,H-SC,7.66,151.0,-0.38,0.68,13.24,XR,ATH,IT
68,SONACOMS,806.63,-0.89,-12.96,60.32,39.54,53112.0,-13108.0,88051.0,-31.96,66.0,M-SC,6.87,220.0,-0.25,0.62,24.16,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.58
2,50,75.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.62
MC    29.29
LC    25.10
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.90
X40      21.13
X40N     11.84
XR        9.65
XY25      9.13
AR        9.01
OX40N     7.71
X200      1.86
SR        0.99
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.03
X-MC    22.86
X-LC    19.96
M-SC    12.06
X-SC     8.13
H-MC     4.71
H-LC     2.97
L-SC     1.40
M-MC     1.38
M-LC     1.19
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.76,-4.82,38.72
IT,13.77,-14.70,73.91
FINANCE,9.45,-14.60,63.02
MISC,7.45,-22.00,71.99
PAINTS,5.98,-7.83,23.85
ELECTRICAL,5.88,-11.44,52.25
INSURANCE,4.42,0.08,35.13
PHARMA,3.86,-1.22,33.61
AUTO,3.35,-20.19,71.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3043466.0,21
AR,1284533.0,10
XR,1281540.0,13
X40,1012912.0,14
X40N,751756.0,9
OX40N,689303.0,10
XY25,333836.0,6
SR,277655.0,2
MH,70883.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3539150.0,25
M-SC,1333395.0,15
X-MC,1209293.0,16
X-LC,903701.0,11
X-SC,755684.0,8
H-MC,405012.0,3
L-SC,260687.0,3
H-LC,132309.0,2
M-LC,121986.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1217888.0      6
           AR         870175.0      5
           XR         783758.0      7
M-SC       XY24       779039.0      6
X-MC       X40        439811.0      7
X-LC       X40        394267.0      5
X-MC       XY24       348468.0      3
H-SC       OX40N      318791.0      4
X-SC       X40N       294896.0      3
M-SC       OX40N      282567.0      5
X-SC       XY24       281954.0      3
H-SC       SR         277655.0      2
X-MC       X40N       246268.0      4
X-LC       XY24       227486.0      2
H-MC       AR         216381.0      2
X-LC       X40N       210592.0      2
H-MC       XY24       188631.0      1
X-SC       X40        178834.0      2
X-MC       XY25       174746.0      2
L-SC       XR         172742.0      2
M-SC       XR         148707.0      2
           AR         123082.0      2
M-LC       XR         121986.0      1
L-SC       OX40N       87945.0      1
H-LC       AR          74895.0      1
X-LC       XY25        71356.0      2
H-SC       MH          70883.0      1
H-LC       X200        57414.0      1
L-MC       XR          54347.0      1
M-MC       XY25        48287.0      1
L-LC       XY25        39447.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 61.0 seconds
